# Drug-Labtest Pairs Retieval and T-Test P-values

## Import Libraries

In [ ]:
import pandas as pd
import datetime
import numpy as np
from scipy.stats import mannwhitneyu
from scipy import stats
from tqdm import tqdm
import os

## Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PARENT='/content/drive/MyDrive/TAU'
DATA = PARENT+'/Datasets/mimiciii/1.4'

In [ ]:
for i in os.walk(os.path.join(DATA)):
    print(i)

('/content/drive/MyDrive/TAU/Datasets/mimiciii/1.4', [], ['CALLOUT.csv.gz', 'ADMISSIONS.csv.gz', 'CAREGIVERS.csv.gz', 'D_ITEMS.csv.gz', 'D_ICD_PROCEDURES.csv.gz', 'D_ICD_DIAGNOSES.csv.gz', 'ICUSTAYS.csv.gz', 'CHARTEVENTS.csv.gz', 'DRGCODES.csv.gz', 'CPTEVENTS.csv.gz', 'DIAGNOSES_ICD.csv.gz', 'D_CPT.csv.gz', 'D_LABITEMS.csv.gz', 'DATETIMEEVENTS.csv.gz', 'INPUTEVENTS_CV.csv.gz', 'INPUTEVENTS_MV.csv.gz', 'LABEVENTS.csv.gz', 'MICROBIOLOGYEVENTS.csv.gz', 'OUTPUTEVENTS.csv.gz', 'PATIENTS.csv.gz', 'README.md', 'PROCEDURES_ICD.csv.gz', 'PRESCRIPTIONS.csv.gz', 'TRANSFERS.csv.gz', 'PROCEDUREEVENTS_MV.csv.gz', 'SERVICES.csv.gz', 'LICENSE.txt', 'NOTEEVENTS.csv.gz', 'SHA256SUMS.txt', 'index.html'])


### Labevents

In [ ]:
labevents = pd.read_csv(os.path.join(DATA, 'LABEVENTS.csv.gz')).dropna()
d_labitems = pd.read_csv(os.path.join(DATA, 'D_LABITEMS.csv.gz')).dropna()

In [ ]:
labevents.shape, d_labitems.shape

((7881769, 9), (585, 6))

In [ ]:
labevents

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
162,443,3,145834.0,50893,2101-10-20 16:40:00,8.2,8.2,mg/dL,abnormal
163,444,3,145834.0,50902,2101-10-20 16:40:00,99,99.0,mEq/L,abnormal
166,447,3,145834.0,50912,2101-10-20 16:40:00,3.2,3.2,mg/dL,abnormal
169,450,3,145834.0,50970,2101-10-20 16:40:00,4.8,4.8,mg/dL,abnormal
170,451,3,145834.0,50971,2101-10-20 16:40:00,5.4,5.4,mEq/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854045,27428430,96443,103219.0,50862,2109-12-30 01:40:00,2.2,2.2,g/dL,abnormal
27854046,27428431,96443,103219.0,50863,2109-12-30 01:40:00,172,172.0,IU/L,abnormal
27854049,27428434,96443,103219.0,50878,2109-12-30 01:40:00,467,467.0,IU/L,abnormal
27854051,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.1,mg/dL,abnormal


In [ ]:
labValues = pd.merge(labevents, d_labitems, on='ITEMID', how='inner')

In [ ]:
labValues.shape

(7876570, 14)

In [ ]:
# subject_id,l.hadm_id, d.label, l.valuenum, l.valueuom, l.charttime
labValues = labValues[['SUBJECT_ID', 'HADM_ID', 'LABEL', 'VALUENUM', 'VALUEUOM', 'CHARTTIME']]

In [ ]:
labValues['CHARTTIME'] = pd.to_datetime(labValues['CHARTTIME'],  format='%Y/%m/%d %H:%M:%S')

In [ ]:
labValues

,SUBJECT_ID,HADM_ID,LABEL,VALUENUM,VALUEUOM,CHARTTIME
0,3,145834.0,"Calcium, Total",8.2,mg/dL,2101-10-20 16:40:00
1,3,145834.0,"Calcium, Total",7.6,mg/dL,2101-10-22 04:00:00
2,3,145834.0,"Calcium, Total",7.1,mg/dL,2101-10-22 21:15:00
3,3,145834.0,"Calcium, Total",7.5,mg/dL,2101-10-23 03:45:00
4,3,145834.0,"Calcium, Total",7.4,mg/dL,2101-10-20 19:59:00
...,...,...,...,...,...,...
7876565,53160,138843.0,Plasma,1.0,%,2194-06-27 13:52:00
7876566,96240,168044.0,Plasma,7.0,%,2105-05-16 16:45:00
7876567,40200,108146.0,Sex Hormone Binding Globulin,12.0,nmol/L,2165-01-10 16:00:00
7876568,58010,185994.0,Sex Hormone Binding Globulin,11.0,nmol/L,2109-02-19 00:16:00


In [ ]:
del labevents, d_labitems

### Chartevents

In [ ]:
chartevents = pd.read_csv(os.path.join(DATA, 'CHARTEVENTS.csv.gz'), nrows=5000000)
d_items = pd.read_csv(os.path.join(DATA, 'D_ITEMS.csv.gz'))

In [ ]:
chartevents.shape, d_items.shape

In [ ]:
print(d_items.columns)
print(chartevents.columns)

In [ ]:
chartValues = pd.merge(d_items[['ITEMID', 'LABEL']], chartevents[['HADM_ID', 'SUBJECT_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'VALUENUM', 'VALUEUOM']], on='ITEMID', how='inner').dropna()

In [ ]:
chartValues['CHARTTIME'] = pd.to_datetime(chartValues['CHARTTIME'],  format='%Y/%m/%d %H:%M:%S')

In [ ]:
chartValues.shape

In [ ]:
# c.subject_id,c.hadm_id,d.label,  c.valuenum, c.valueuom ,d.category, c.charttime
chartValues

In [ ]:
del chartevents, d_items

### Prescriptions

In [ ]:
prescriptions = pd.read_csv(os.path.join(DATA, 'PRESCRIPTIONS.csv.gz'), nrows=1000000).dropna()

In [ ]:
prescriptions.shape

(360680, 19)

In [ ]:
prescriptions.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE', 'ENDDATE',
       'DRUG_TYPE', 'DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC',
       'FORMULARY_DRUG_CD', 'GSN', 'NDC', 'PROD_STRENGTH', 'DOSE_VAL_RX',
       'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE'],
      dtype='object')

In [ ]:
print(prescriptions['DRUG'].unique().shape)
print(prescriptions['FORMULARY_DRUG_CD'].unique().shape)

In [ ]:
# subject_id,hadm_id,startdate,drug,dose_val_rx
prescriptions = prescriptions[['SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'DRUG', 'DOSE_VAL_RX', 'FORMULARY_DRUG_CD', 'DOSE_UNIT_RX']]
prescriptions['STARTDATE'] = pd.to_datetime(prescriptions['STARTDATE'],  format='%Y/%m/%d %H:%M:%S')
prescriptions

In [ ]:
drugPlot = prescriptions['DRUG'].value_counts(sort=True)

In [ ]:
drugPlot.plot(kind='barh')

In [ ]:
del drugPlot

### Top 10 Drugs

In [ ]:
top100Drugs = drugPlot.iloc[:100]

In [ ]:
for drug in top100Drugs.iloc[:10].items():
    print(drug)

### Admissions

In [ ]:
admissions = pd.read_csv(os.path.join(DATA, 'ADMISSIONS.csv.gz'))

In [ ]:
# subject_id,hadm_id
admissions = admissions[['SUBJECT_ID', 'HADM_ID']]
admissions

### Input Events

In [ ]:
inputevents_cv = pd.read_csv(os.path.join(DATA, 'INPUTEVENTS_CV.csv.gz'), nrows=500000)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
inputevents_mv = pd.read_csv(os.path.join(DATA, 'INPUTEVENTS_MV.csv.gz'), nrows=500000)

In [ ]:
inputevents_cv.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID',
       'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM', 'STORETIME', 'CGID',
       'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE', 'ORIGINALAMOUNT',
       'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE', 'ORIGINALRATEUOM',
       'ORIGINALSITE'],
      dtype='object')

In [ ]:
inputevents_cv['ORIGINALAMOUNT'].value_counts()

100.0     311958
1000.0         2
Name: ORIGINALAMOUNT, dtype: int64

In [ ]:
inputevents_mv.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME',
       'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM', 'STORETIME', 'CGID',
       'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME',
       'SECONDARYORDERCATEGORYNAME', 'ORDERCOMPONENTTYPEDESCRIPTION',
       'ORDERCATEGORYDESCRIPTION', 'PATIENTWEIGHT', 'TOTALAMOUNT',
       'TOTALAMOUNTUOM', 'ISOPENBAG', 'CONTINUEINNEXTDEPT', 'CANCELREASON',
       'STATUSDESCRIPTION', 'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY',
       'COMMENTS_DATE', 'ORIGINALAMOUNT', 'ORIGINALRATE'],
      dtype='object')

In [ ]:
inputevents_mv['ORDERCATEGORYNAME'].value_counts()

01-Drips                    196312
02-Fluids (Crystalloids)     76453
08-Antibiotics (IV)          59048
14-Oral/Gastric Intake       51766
05-Med Bolus                 33985
13-Enteral Nutrition         18647
06-Insulin (Non IV)          15424
11-Prophylaxis (Non IV)      13991
10-Prophylaxis (IV)          10121
03-IV Fluid Bolus             9305
07-Blood Products             5369
12-Parenteral Nutrition       3146
16-Pre Admission              2787
09-Antibiotics (Non IV)       1861
04-Fluids (Colloids)          1768
15-Supplements                  17
Name: ORDERCATEGORYNAME, dtype: int64

### Item

In [ ]:
d_item = pd.read_csv(os.path.join(DATA, 'D_ITEMS.csv.gz'))

In [ ]:
d_item[d_item['CATEGORY']=='Antibiotics']

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9859,14019,225848,Caspofungin,Caspofungin,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN
9860,14020,225850,Cefazolin,Cefazolin,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN
9861,14021,225851,Cefepime,Cefepime,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN
9862,14022,225853,Ceftazidime,Ceftazidime,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN
9863,14023,225855,Ceftriaxone,Ceftriaxone,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN
9864,14024,225857,Chloroquine,Chloroquine,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN
9865,14025,225859,Ciprofloxacin,Ciprofloxacin,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN
9866,14026,225860,Clindamycin,Clindamycin,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN
9867,14027,225862,Colistin,Colistin,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN
9868,14028,225863,Daptomycin,Daptomycin,metavision,inputevents_mv,Antibiotics,dose,Solution,NaN


### Item <> Input events MV

In [ ]:
ditem_inputevents_mv = pd.merge(inputevents_mv, d_item, on='ITEMID', how='inner')

In [ ]:
ditem_inputevents_mv.columns

Index(['ROW_ID_x', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME',
       'ENDTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM',
       'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME',
       'SECONDARYORDERCATEGORYNAME', 'ORDERCOMPONENTTYPEDESCRIPTION',
       'ORDERCATEGORYDESCRIPTION', 'PATIENTWEIGHT', 'TOTALAMOUNT',
       'TOTALAMOUNTUOM', 'ISOPENBAG', 'CONTINUEINNEXTDEPT', 'CANCELREASON',
       'STATUSDESCRIPTION', 'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY',
       'COMMENTS_DATE', 'ORIGINALAMOUNT', 'ORIGINALRATE', 'ROW_ID_y', 'LABEL',
       'ABBREVIATION', 'DBSOURCE', 'LINKSTO', 'CATEGORY', 'UNITNAME',
       'PARAM_TYPE', 'CONCEPTID'],
      dtype='object')

In [ ]:
inputevents_mv_1 = ditem_inputevents_mv[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'UNITNAME', 'ORDERCATEGORYNAME', 'LABEL', 'CATEGORY', 'PARAM_TYPE']]
inputevents_mv_1

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,UNITNAME,ORDERCATEGORYNAME,LABEL,CATEGORY,PARAM_TYPE
0,27063,139787,223259.0,2133-02-05 06:29:00,2133-02-05 08:45:00,225166,6.774532,mEq,mEq,02-Fluids (Crystalloids),Potassium Chloride,Medications,Solution
1,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,225166,2.813300,mEq,mEq,02-Fluids (Crystalloids),Potassium Chloride,Medications,Solution
2,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 07:03:00,225166,4.433333,mEq,mEq,02-Fluids (Crystalloids),Potassium Chloride,Medications,Solution
3,27063,139787,223259.0,2133-02-05 09:43:00,2133-02-05 12:30:00,225166,8.360021,mEq,mEq,02-Fluids (Crystalloids),Potassium Chloride,Medications,Solution
4,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 05:35:00,225166,0.050060,mEq,mEq,02-Fluids (Crystalloids),Potassium Chloride,Medications,Solution
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,65382,163918,288487.0,2132-12-09 02:35:00,2132-12-09 02:36:00,227692,0.004046,mg,mg,01-Drips,Isuprel,Medications,Solution
499996,60970,163433,274168.0,2132-02-08 05:37:00,2132-02-08 07:15:00,225148,249.999999,mg,mg,01-Drips,Bivalirudin (Angiomax),Medications,Solution
499997,45426,166752,236992.0,2144-01-06 17:33:00,2144-01-07 00:16:00,226022,134.333336,ml,mL,13-Enteral Nutrition,Impact (3/4),Nutrition - Enteral,Solution
499998,54586,122570,222417.0,2168-03-09 11:00:00,2168-03-09 12:00:00,225173,7500.000000,ml,mL,07-Blood Products,Cell Saver,Blood Products/Colloids,Solution


In [ ]:
top200Meds = inputevents_mv_1['LABEL'].value_counts()[:200]

In [ ]:
inputevents_mv_1['CATEGORY'].value_counts()

Fluids/Intake              242772
Medications                196463
Antibiotics                 31382
Nutrition - Enteral         18661
Blood Products/Colloids      7929
Nutrition - Parenteral       2790
Nutrition - Supplements         3
Name: CATEGORY, dtype: int64

### Item <> Input events CV

In [ ]:
ditem_inputevents_cv = pd.merge(inputevents_cv, d_item, on='ITEMID', how='inner')

In [ ]:
ditem_inputevents_cv['LABEL'].value_counts()

D5W          311960
Po Intake    188040
Name: LABEL, dtype: int64

### Prescription - Inputevents MV

In [ ]:
prescriptions.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE', 'ENDDATE',
       'DRUG_TYPE', 'DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC',
       'FORMULARY_DRUG_CD', 'GSN', 'NDC', 'PROD_STRENGTH', 'DOSE_VAL_RX',
       'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE'],
      dtype='object')

In [ ]:
presc_inputevents_mv = pd.merge(inputevents_mv_1, prescriptions, on='ICUSTAY_ID', how='inner')

In [ ]:
presc_inputevents_mv.shape

(1636222, 31)

In [ ]:
presc_inputevents_mv.columns

Index(['SUBJECT_ID_x', 'HADM_ID_x', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME',
       'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'UNITNAME', 'ORDERCATEGORYNAME',
       'LABEL', 'CATEGORY', 'PARAM_TYPE', 'ROW_ID', 'SUBJECT_ID_y',
       'HADM_ID_y', 'STARTDATE', 'ENDDATE', 'DRUG_TYPE', 'DRUG',
       'DRUG_NAME_POE', 'DRUG_NAME_GENERIC', 'FORMULARY_DRUG_CD', 'GSN', 'NDC',
       'PROD_STRENGTH', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'FORM_VAL_DISP',
       'FORM_UNIT_DISP', 'ROUTE'],
      dtype='object')

In [ ]:
presc_inputevents_mv[['STARTTIME', 'ENDTIME', 'STARTDATE', 'ENDDATE']]

,STARTTIME,ENDTIME,STARTDATE,ENDDATE
0,2181-02-27 16:00:00,2181-02-27 17:00:00,2181-02-25 00:00:00,2181-02-25 00:00:00
1,2181-02-27 16:00:00,2181-02-27 17:00:00,2181-02-25 00:00:00,2181-02-26 00:00:00
2,2181-02-27 16:00:00,2181-02-27 17:00:00,2181-02-25 00:00:00,2181-02-26 00:00:00
3,2181-02-27 16:00:00,2181-02-27 17:00:00,2181-02-25 00:00:00,2181-02-26 00:00:00
4,2181-02-27 16:00:00,2181-02-27 17:00:00,2181-02-25 00:00:00,2181-02-26 00:00:00
...,...,...,...,...
1636217,2182-01-07 08:22:00,2182-01-07 08:23:00,2182-01-07 00:00:00,2182-01-08 00:00:00
1636218,2182-01-07 08:22:00,2182-01-07 08:23:00,2182-01-07 00:00:00,2182-01-08 00:00:00
1636219,2182-01-07 08:22:00,2182-01-07 08:23:00,2182-01-08 00:00:00,2182-01-08 00:00:00
1636220,2182-01-07 08:22:00,2182-01-07 08:23:00,2182-01-08 00:00:00,2182-01-08 00:00:00


In [ ]:
presc_inputevents_mv[['SUBJECT_ID_x', 'HADM_ID_x', 'ICUSTAY_ID','LABEL', 'CATEGORY', 'DRUG', 'DRUG_TYPE', 'PROD_STRENGTH', 'AMOUNT', 'AMOUNTUOM', 'UNITNAME', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'FORM_VAL_DISP',
       'FORM_UNIT_DISP', 'ROUTE']]

,SUBJECT_ID_x,HADM_ID_x,ICUSTAY_ID,LABEL,CATEGORY,DRUG,DRUG_TYPE,PROD_STRENGTH,AMOUNT,AMOUNTUOM,UNITNAME,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,5060,117340,231947.0,Potassium Chloride,Medications,Diazepam,MAIN,10mg/2mL Syringe,20.0,mEq,mEq,10,mg,1,SYR,IV
1,5060,117340,231947.0,Potassium Chloride,Medications,Diazepam,MAIN,10mg/2mL Syringe,20.0,mEq,mEq,15,mg,1.5,SYR,IV
2,5060,117340,231947.0,Potassium Chloride,Medications,Acetaminophen,MAIN,325mg Tablet,20.0,mEq,mEq,650,mg,2,TAB,PO
3,5060,117340,231947.0,Potassium Chloride,Medications,Thiamine,MAIN,100mg Tablet,20.0,mEq,mEq,100,mg,1,TAB,PO
4,5060,117340,231947.0,Potassium Chloride,Medications,Magnesium Sulfate,MAIN,2 g / 50 mL Premix Bag,20.0,mEq,mEq,2,gm,1,BAG,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1636217,5060,196749,219030.0,Famotidine (Pepcid),Medications,Diazepam,MAIN,5 mg Tab,1.0,dose,dose,10,mg,2,TAB,PO
1636218,5060,196749,219030.0,Famotidine (Pepcid),Medications,Tuberculin Protein,MAIN,0.1mL Syringe,1.0,dose,dose,0.1,mL,1,SYR,ID
1636219,5060,196749,219030.0,Famotidine (Pepcid),Medications,Diazepam,MAIN,10 mg Tab,1.0,dose,dose,10,mg,1,TAB,PO
1636220,5060,196749,219030.0,Famotidine (Pepcid),Medications,Diazepam,MAIN,10 mg Tab,1.0,dose,dose,10,mg,1,TAB,PO


In [ ]:
presc_inputevents_mv

,SUBJECT_ID_x,HADM_ID_x,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,UNITNAME,ORDERCATEGORYNAME,...,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,5060,117340,231947.0,2181-02-27 16:00:00,2181-02-27 17:00:00,225166,20.0,mEq,mEq,02-Fluids (Crystalloids),...,Diazepam,DIAZ10I,003762,4.091273e+08,10mg/2mL Syringe,10,mg,1,SYR,IV
1,5060,117340,231947.0,2181-02-27 16:00:00,2181-02-27 17:00:00,225166,20.0,mEq,mEq,02-Fluids (Crystalloids),...,Diazepam,DIAZ10I,003762,4.091273e+08,10mg/2mL Syringe,15,mg,1.5,SYR,IV
2,5060,117340,231947.0,2181-02-27 16:00:00,2181-02-27 17:00:00,225166,20.0,mEq,mEq,02-Fluids (Crystalloids),...,Acetaminophen,ACET325,004489,1.828448e+08,325mg Tablet,650,mg,2,TAB,PO
3,5060,117340,231947.0,2181-02-27 16:00:00,2181-02-27 17:00:00,225166,20.0,mEq,mEq,02-Fluids (Crystalloids),...,Thiamine,THIA100,002451,1.820047e+08,100mg Tablet,100,mg,1,TAB,PO
4,5060,117340,231947.0,2181-02-27 16:00:00,2181-02-27 17:00:00,225166,20.0,mEq,mEq,02-Fluids (Crystalloids),...,Magnesium Sulfate,MAG2PM,016546,4.096729e+08,2 g / 50 mL Premix Bag,2,gm,1,BAG,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1636217,5060,196749,219030.0,2182-01-07 08:22:00,2182-01-07 08:23:00,225907,1.0,dose,dose,11-Prophylaxis (Non IV),...,Diazepam,DIAZ5,003768,5.107903e+10,5 mg Tab,10,mg,2,TAB,PO
1636218,5060,196749,219030.0,2182-01-07 08:22:00,2182-01-07 08:23:00,225907,1.0,dose,dose,11-Prophylaxis (Non IV),...,Tuberculin Protein,PPD,009700,4.928108e+10,0.1mL Syringe,0.1,mL,1,SYR,ID
1636219,5060,196749,219030.0,2182-01-07 08:22:00,2182-01-07 08:23:00,225907,1.0,dose,dose,11-Prophylaxis (Non IV),...,Diazepam,DIAZ10,003766,5.107903e+10,10 mg Tab,10,mg,1,TAB,PO
1636220,5060,196749,219030.0,2182-01-07 08:22:00,2182-01-07 08:23:00,225907,1.0,dose,dose,11-Prophylaxis (Non IV),...,Diazepam,DIAZ10,003766,5.107903e+10,10 mg Tab,10,mg,1,TAB,PO


In [ ]:
presc_inputevents_mv[['DRUG', 'STARTDATE', 'STARTTIME', 'STORETIME', 'ENDTIME', 'ENDDATE']]

,DRUG,STARTDATE,STARTTIME,STORETIME,ENDTIME,ENDDATE
0,Diazepam,2181-02-25 00:00:00,2181-02-27 11:30:00,2181-02-27 13:50:00,2181-02-28 07:30:00,2181-02-25 00:00:00
1,Diazepam,2181-02-25 00:00:00,2181-02-27 11:30:00,2181-02-27 13:50:00,2181-02-28 07:30:00,2181-02-26 00:00:00
2,Acetaminophen,2181-02-25 00:00:00,2181-02-27 11:30:00,2181-02-27 13:50:00,2181-02-28 07:30:00,2181-02-26 00:00:00
3,Thiamine,2181-02-25 00:00:00,2181-02-27 11:30:00,2181-02-27 13:50:00,2181-02-28 07:30:00,2181-02-26 00:00:00
4,Magnesium Sulfate,2181-02-25 00:00:00,2181-02-27 11:30:00,2181-02-27 13:50:00,2181-02-28 07:30:00,2181-02-26 00:00:00
...,...,...,...,...,...,...
1274506,Insulin,2156-02-27 00:00:00,2156-02-26 07:30:00,2156-02-26 07:36:00,2156-02-26 11:15:00,2156-03-02 00:00:00
1274507,Pantoprazole,2156-02-27 00:00:00,2156-02-26 07:30:00,2156-02-26 07:36:00,2156-02-26 11:15:00,2156-03-02 00:00:00
1274508,HydrOXYzine,2156-02-27 00:00:00,2156-02-26 07:30:00,2156-02-26 07:36:00,2156-02-26 11:15:00,2156-03-02 00:00:00
1274509,Insulin,2156-02-27 00:00:00,2156-02-26 07:30:00,2156-02-26 07:36:00,2156-02-26 11:15:00,2156-03-02 00:00:00


In [ ]:
presc_inputevents_mv['STORETIME']

0          2181-02-27 13:50:00
1          2181-02-27 13:50:00
2          2181-02-27 13:50:00
3          2181-02-27 13:50:00
4          2181-02-27 13:50:00
                  ...         
1274506    2156-02-26 07:36:00
1274507    2156-02-26 07:36:00
1274508    2156-02-26 07:36:00
1274509    2156-02-26 07:36:00
1274510    2156-02-26 07:36:00
Name: STORETIME, Length: 1274511, dtype: object

In [ ]:
inputmv_chartevents = pd.merge(chartevents, inputevents_mv, on=['ICUSTAY_ID'], how='inner')

In [ ]:
inputmv_chartevents

## Data Preprocessing

In [ ]:
def remove_multiple_admissions(df):
    """
    Removes hospital admissions that occur more than once for the same patient
  
    Parameters:
    df (DataFrame): Takes in dataframe with multiple hospital admissions
  
    Returns:
    Dataframe: Returns dataframe with multiple hospital admissions removed
    """
    first_admissions = admissions
    first_admissions = first_admissions.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
    df = df[df['HADM_ID'].isin(first_admissions['HADM_ID'])]
    return df

In [ ]:
lab_measurements = labValues

In [ ]:
top100_drugs = pd.DataFrame(top100Drugs, columns=['DRUG']).reset_index()
top100_drugs.rename(columns = {'index':'DRUG', 'DRUG':'COUNT'}, inplace = True)
top100_drugs

,DRUG,COUNT
0,Furosemide,85851
1,Insulin,84420
2,Potassium Chloride,63168
3,Metoprolol,51486
4,Acetaminophen,47726
...,...,...
95,Clopidogrel Bisulfate,2956
96,Omeprazole,2862
97,traZODONE,2735
98,HydrALAZINE HCl,2708


In [ ]:
patient_presc = prescriptions

In [ ]:
patient_presc = remove_multiple_admissions(patient_presc)
patient_presc = patient_presc[patient_presc['DRUG'].isin(top100_drugs['DRUG'])]

In [ ]:
patient_presc['STARTDATE'] += datetime.timedelta(hours=12)
patient_presc

,SUBJECT_ID,HADM_ID,STARTDATE,DRUG,DOSE_VAL_RX,FORMULARY_DRUG_CD,DOSE_UNIT_RX
30,13,143045,2167-01-10 12:00:00,Midazolam HCl,2,MIDA2I,mg
32,13,143045,2167-01-10 12:00:00,Lorazepam,1,LORA2I,mg
33,13,143045,2167-01-10 12:00:00,Magnesium Sulfate,2,MAGS1I,gm
34,13,143045,2167-01-10 12:00:00,Potassium Chloride,40,MICROK10,mEq
35,13,143045,2167-01-10 12:00:00,Ketorolac,15,KETO15I,mg
...,...,...,...,...,...,...,...
4156445,98887,121032,2144-09-07 12:00:00,PredniSONE,40,PRED20,mg
4156446,98887,121032,2144-09-07 12:00:00,Ipratropium Bromide Neb,1,IPRA2H,NEB
4156447,98887,121032,2144-09-07 12:00:00,HYDROmorphone (Dilaudid),0.12,HYDR20/100NS,mg
4156448,98887,121032,2144-09-07 12:00:00,Docusate Sodium,100,DOCU100,mg


In [ ]:
lab_measurements = lab_measurements[lab_measurements.duplicated(subset=['SUBJECT_ID','LABEL'],keep=False)]
lab_measurements = lab_measurements[lab_measurements['HADM_ID'].isin(patient_presc['HADM_ID'])]

## Generating Lab Test<>Drug Pairings

In [ ]:
def labpairing(drugname, prescdf, labdf, labname):
    """Pairs the drug input with each lab test

    Parameters:
    drugname (String): Drug Name
    prescdf (DataFrame): Dataframe containing the prescription data
    labdf (DataFrame): Dataframe containing the lab measurement data
    labname (DataFrame): Lab Test Name
    Returns:
    DataFrame: Contains all the rows of values and times for that particular drug lab apir
  
    """
    # Select patients who have taken the drug
    prescdf = prescdf[prescdf['DRUG']==drugname]
    prescdf = prescdf.drop_duplicates(subset=['SUBJECT_ID'], keep='first')

    # Select lab measurements of patients who have taken the drug
    labdf = labdf[labdf['HADM_ID'].isin(prescdf['HADM_ID'])]

    # Selects the lab measurement entered
    drug_lab_specific = labdf[labdf['LABEL']==labname]
    mergeddf = pd.merge(drug_lab_specific, prescdf, on=['HADM_ID','SUBJECT_ID'])

    # Get time from prescription and choose before and after lab measurements (within 24hrs=1day)
    mergeddf['timeFromPrescription'] = mergeddf['CHARTTIME'] - mergeddf['STARTDATE']
    mergeddf = mergeddf[(mergeddf['timeFromPrescription']>datetime.timedelta(days=-1)) & (mergeddf['timeFromPrescription']<datetime.timedelta(days=1))]
    posmergeddf = mergeddf.loc[mergeddf.timeFromPrescription > datetime.timedelta(days=0)]
    negmergeddf = mergeddf.loc[mergeddf.timeFromPrescription < datetime.timedelta(days=0)]
    
    # Only keep values for which we have both before and after
    posmergeddf = posmergeddf[posmergeddf['HADM_ID'].isin(negmergeddf['HADM_ID'])]
    negmergeddf = negmergeddf[negmergeddf['HADM_ID'].isin(posmergeddf['HADM_ID'])]

    #Select the values closest to 0
    posmergeddf = posmergeddf.loc[posmergeddf.groupby('SUBJECT_ID').timeFromPrescription.idxmin()]
    negmergeddf = negmergeddf.loc[negmergeddf.groupby('SUBJECT_ID').timeFromPrescription.idxmax()]
    finaldf = negmergeddf.merge(posmergeddf,on=['HADM_ID','SUBJECT_ID'])
    
    return finaldf

## Performing Mannwhitney Test and T-Test

In [ ]:
def postprocessing(df):
    """Gets the mean, standard deviation, mann whitney and t-test p values. Converts time delta to hours
  
    Parameters:
    df (DataFrame): Dataframe containing before and after lab test values and time values
    Returns:
    List:Containing mean, standard deviation, mann whitney and t-test p values and count
  
    """
    df['timeFromPrescription_x'] = pd.to_numeric(df['timeFromPrescription_x'].dt.seconds)
    df['timeFromPrescription_x']/=3600
    df['timeFromPrescription_y'] = pd.to_numeric(df['timeFromPrescription_y'].dt.seconds)
    df['timeFromPrescription_y']/=3600
    df_before_mean = df['VALUENUM_x'].mean()
    df_after_mean = df['VALUENUM_y'].mean()
    df_before_std = df['VALUENUM_x'].std()
    df_after_std = df['VALUENUM_y'].std()
    df_before_time_mean = df['timeFromPrescription_x'].mean()
    df_after_time_mean = df['timeFromPrescription_y'].mean()
    df_before_time_std = df['timeFromPrescription_x'].std()
    df_after_time_std = df['timeFromPrescription_y'].std()
    mannwhitneypvalue = mannwhitneyu(df['VALUENUM_x'], df['VALUENUM_y'])[1]
    ttestpvalue = stats.ttest_ind(df['VALUENUM_x'], df['VALUENUM_y'])[1]
    lengthofdf = len(df)
    csvrow=[lengthofdf,df_before_mean,df_before_std,df_before_time_mean,df_before_time_std,df_after_mean,df_after_std,df_after_time_mean,df_after_time_std,mannwhitneypvalue,ttestpvalue]
    return csvrow

 

In [ ]:
drug_lab = labpairing('Furosemide', patient_presc, lab_measurements, 'Calcium, Total')

## Final Results

In [ ]:
def comp_analysis(lab_measurements, top100_drugs, n_druglab_pairs = 25, n_drugs=None):
    res = pd.DataFrame(columns=['Medication Name','Feature Name','Number of patients','Feature Before(mean)','Feature Before(std)','Time Before(mean)','Time Before(std)','Feature After(mean)','Feature After(std)','Time After(mean)','Time After(std)','Mannwhitney-pvalue','Ttest-pvalue'])
    uniqueLabTests = lab_measurements.LABEL.unique()

    for i, drug in enumerate(top100_drugs['DRUG']): 
        if n_drugs is not None and i>=n_drugs:
            break
        print(i, ' Drug: ', drug)
        for j in tqdm(range(uniqueLabTests.shape[0])):
            labTest = uniqueLabTests[j]
            drug_lab=labpairing(drug, patient_presc, lab_measurements, labTest)
            if(len(drug_lab) > n_druglab_pairs): 
                csvrow=postprocessing(drug_lab)
                csvrow.insert(0, drug) 
                csvrow.insert(1, labTest)
                res.loc[len(res)] = csvrow
    return res

# Potassium Chloride Alanine Aminotransferase (ALT)
# Insulin Alanine Aminotransferase (ALT)

### 1630 Lab Test Pairs

In [ ]:
csvdf = pd.read_csv(os.path.join(PARENT, 'Drug-Labtest_1630_Pairs.csv'))
csvdf

,Medication Name,Feature Name,Number of patients,Feature Before(mean),Feature Before(std),Time Before(mean),Time Before(std),Feature After(mean),Feature After(std),Time After(mean),Time After(std),Mannwhitney-pvalue,Ttest-pvalue
0,Furosemide,"Calcium, Total",3689,7.865194,0.596117,15.318731,3.130199,7.895690,0.767306,12.782805,5.543262,4.869298e-04,0.056653
1,Insulin,"Calcium, Total",4723,7.805293,0.586089,15.372041,3.264240,7.849037,0.591139,12.703472,5.666672,1.005113e-07,0.000306
2,Potassium Chloride,"Calcium, Total",3051,7.837234,0.517342,15.812040,3.026666,7.878663,0.519142,13.507331,5.619706,1.045076e-05,0.001803
3,Metoprolol,"Calcium, Total",2076,7.865896,0.564997,15.530210,3.122839,7.880250,0.555806,13.623627,5.225266,9.761628e-02,0.409290
4,Acetaminophen,"Calcium, Total",3972,7.787941,0.561434,15.379435,3.287844,7.839300,0.563105,13.225097,5.562219,2.322292e-10,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625,Sodium Bicarbonate,PTT,413,55.798063,25.284375,16.674899,4.949476,54.140436,24.302883,8.983535,6.175378,4.808567e-02,0.337058
1626,Phytonadione,PTT,580,51.215069,21.058976,16.234310,4.000901,52.952241,25.770537,10.717529,6.350503,3.275688e-01,0.208976
1627,Neostigmine,PTT,229,46.417031,16.257331,11.809389,7.175490,45.053275,21.336951,11.478384,5.472444,7.425724e-06,0.442089
1628,Ipratropium Bromide MDI,PTT,405,58.165185,25.123963,16.967490,3.866947,56.378272,23.154625,9.598848,6.053865,2.276299e-01,0.292876


### Top 10 Most used Drugs with all lab tests

In [ ]:
res = comp_analysis(lab_measurements, top100_drugs, 100, 10)
# # Potassium Chloride Alanine Aminotransferase (ALT)
# # Insulin Alanine Aminotransferase (ALT)

0  Drug:  Furosemide


100%|██████████| 192/192 [02:21<00:00,  1.35it/s]


1  Drug:  Insulin


100%|██████████| 192/192 [02:26<00:00,  1.31it/s]


2  Drug:  Potassium Chloride


...

100%|██████████| 192/192 [01:51<00:00,  1.72it/s]


8  Drug:  Heparin


100%|██████████| 192/192 [02:04<00:00,  1.54it/s]


9  Drug:  Docusate Sodium


100%|██████████| 192/192 [01:51<00:00,  1.72it/s]


In [ ]:
# res['Medication Name'] = res['Medication Name'].apply(lambda x : top100_drugs.iloc[x]['DRUG'])
res

,Medication Name,Feature Name,Number of patients,Feature Before(mean),Feature Before(std),Time Before(mean),Time Before(std),Feature After(mean),Feature After(std),Time After(mean),Time After(std),Mannwhitney-pvalue,Ttest-pvalue
0,Furosemide,"Calcium, Total",3689,7.865194,0.596117,15.318731,3.130199,7.895690,0.767306,12.782805,5.543262,0.000487,0.056653
1,Furosemide,Chloride,1968,106.545224,9.555180,15.720207,3.192894,106.237297,9.658326,11.904870,6.130098,0.015742,0.314740
2,Furosemide,Creatinine,3930,2.309898,1.588903,16.197752,2.726862,2.295802,1.562848,12.027998,6.323819,0.127153,0.691735
3,Furosemide,Phosphate,2442,3.661261,2.088406,15.352662,3.674798,3.619165,2.088854,12.280774,5.934769,0.479578,0.481294
4,Furosemide,Potassium,414,4.069082,1.339911,14.442351,5.496923,4.050725,1.323301,10.155837,6.765043,0.454535,0.842826
...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,Docusate Sodium,Calculated Total CO2,480,26.868750,8.603385,16.759549,6.344186,27.554167,8.572738,7.386979,6.572813,0.040627,0.216607
422,Docusate Sodium,"Potassium, Whole Blood",107,4.128972,1.292869,12.129128,8.536525,4.085047,1.282598,8.191900,7.232208,0.322723,0.803220
423,Docusate Sodium,Phenytoin,134,8.267164,6.803181,15.278731,3.094043,8.167164,5.349426,15.054851,3.909030,0.294080,0.893697
424,Docusate Sodium,Troponin T,271,1.312568,2.307688,15.977245,4.758200,1.287048,2.276174,10.673063,6.441718,0.370285,0.896922


In [ ]:
# res.to_csv(os.path.join(PARENT, 'Drug-Labtest_Pairs_Top10Drugs.csv'))
res_analysis = pd.read_csv(os.path.join(PARENT, 'Drug-Labtest_Pairs_Top10Drugs.csv'))